In [42]:
# 작동원리: 꼭 주석 다 읽어보세요

# 1. 밑에 바꿀 부분을 설정해준다
# 2. 첫번째 스크립트를 실행하고 끝까지 돌린다
# 3. 다 돌렸으면("Annotation done." 이라 떠야함) 두번째 스크립트를 돌린다
# 4. 사진이 다 사라질때까지 2번과 3번을 반복한다



# pip install pigeon-jupyter
from pigeon import annotate
from IPython.display import display, Image
import os
import pandas as pd

####################################
# 바꿀 부분

# 이미지 위치
PATH="/opt/ml/test_images"
# 저장할 위치
SAVE_PATH="/opt/ml/save_image"
# 레이블
NAME="광어"
# 한번에 처리할 이미지들 (중간에 멈추면 다시 해야하기 때문에 크지 않게 설정)
NUM=100
####################################
if not os.path.exists(SAVE_PATH):
  os.makedirs(SAVE_PATH)

files=[]
for file in os.listdir(PATH):
  files.append(os.path.join(PATH,file))
  if len(files)>NUM:
    break

annotations = annotate(
  files,
  options=[NAME],
  display_fn=lambda filename: display(Image(filename))
)

HTML(value='0 examples annotated, 3 examples left')

Output()

Annotation done.


In [43]:
# 위에서 Annotation done. 이라고 뜨기 전까지는 하지 않기
import shutil
import secrets
if os.path.exists(SAVE_PATH+"/label.csv"):
    df=pd.read_csv(SAVE_PATH+"/label.csv")
else:
    df = pd.DataFrame(columns=["img_name","label"])

# starting number of file
idx=0
for v,n in annotations:
    new_name=secrets.token_hex(30)+os.path.splitext(v)[1]
    shutil.move(v,os.path.join(SAVE_PATH,new_name))
    df_new=pd.DataFrame({"img_name":[new_name],"label":[NAME]})
    df=pd.concat((df,df_new),ignore_index=True)
    idx+=1
df.to_csv(SAVE_PATH+"/label.csv",index=False)

# remove skipped file
for f in files:
    if os.path.exists(f):
        os.remove(f)